In [1]:
from PyPDF2 import PdfReader
import os
import regex
import datetime as dt
from dateutil.relativedelta import relativedelta

In [2]:
# abrir PDF

file = 'test_pdfs/CPU.pdf'
filepath = os.path.abspath(os.path.join(os.getcwd(), file))

In [3]:
# extrair conteúdo do PDF com PyPDF2

reader = PdfReader(filepath)

text = [reader.pages[page].extract_text() for page in range(len(reader.pages))]
text = ' '.join(text)
text = text.replace('\n','')

In [4]:
text

'060318 - SANTO ANTONIO DOS OLIVAIS - U - 12626 - A  CADERNETA PREDIAL URBANA SERVIÇO DE FINANÇAS: 0728 - COIMBRA-1.  IDENTIFICAÇÃO DO PRÉDIO DISTRITO: 06 - COIMBRA CONCELHO: 03 - COIMBRA FREGUESIA: 18 - SANTO ANTONIO DOS OLIVAIS  ARTIGO MATRICIAL: 12626 NIP:  Descrito na C.R.P. de : COIMBRA sob o registo nº: 3662 TEVE ORIGEM NOS ARTIGOS DISTRITO: 06 - COIMBRA CONCELHO: 03 - COIMBRA FREGUESIA: 18 - SANTO ANTONIO DOS OLIVAIS Tipo: URBANO  Artigo: 9396 LOCALIZAÇÃO DO PRÉDIO Av./Rua/Praça: Rua Padre António Nogueira, Urb. Quinta da Lomba Lote: 35 Lugar: Chão do Bispo Código Postal: 3030-416 COIMBRA CONFRONTAÇÕES Norte: . Sul: . Nascente: . Poente: . DESCRIÇÃO DO PRÉDIO Tipo de Prédio: Prédio em Regime de Prop. Horiz.  Nº de pisos do artigo: 5 ÁREAS (em  m²) Área total do terreno: 555,0000 m² Área de implantação do edifício: 555,0000 m² Área bruta privativa total: 83,0000 m² Área de terreno integrante das fracções: 0,0000 m² FRACÇÃO AUTÓNOMA:  A  LOCALIZAÇÃO DA FRACÇÃO Av./Rua/Praça: Rua P

In [11]:
### extrair informação relevante da CPU ###

distrito = regex.findall('(?<=DISTRITO: )\d+', text)
concelho = regex.findall('(?<=CONCELHO: )\d+', text)
freguesia = regex.findall('(?<=FREGUESIA: )\d+', text)
distrito_concelho = distrito[0] + concelho[0]
distrito_concelho_freguesia = distrito_concelho + freguesia[0]

ano_inscricao = regex.findall('(?<=Ano de inscrição na matriz: )(.*)(?= Valor patrimonial actual)', text)
ano_inscricao = int(ano_inscricao[0])

data_avaliacao = regex.findall('(?<=Avaliada em : )(.*)', text)
data_avaliacao = data_avaliacao[0].split()[0]
data_avaliacao = dt.datetime.strptime(data_avaliacao, '%Y/%m/%d')
data_avaliacao = data_avaliacao.date()

ext_calc = regex.findall('(?<=Vc x A x Ca x Cl x Cq x Cv )(.*)(?= Vt = valor patrimonial tributário)', text)
ext_calc = ext_calc[0].split()

ValueError: time data '2020/11/27' does not match format '%Y-%m-%d'

In [6]:
### extrair o VPT existente ###

VPT_existente = float(ext_calc[0].replace('.', '').replace(',', '.'))

In [7]:
### calcular o VPT em caso de reavaliação ###

# valor de construção em 2022 (última actualização deste código)

Vc = 640.00

# extrair coeficientes da CPU

A = float(ext_calc[4].replace('.', '').replace(',', '.'))
Ca = float(ext_calc[6].replace('.', '').replace(',', '.'))
Cl = float(ext_calc[8].replace('.', '').replace(',', '.'))
Cq = float(ext_calc[10].replace('.', '').replace(',', '.'))

# calcular o coeficiente de vetustez

idade = dt.date.today().year - ano_inscricao

if idade < 2:
    Cv = 1.0
elif idade >= 2 and idade <= 8:
    Cv = 0.9
elif idade >= 9 and idade <= 15:
    Cv = 0.85
elif idade >= 16 and idade <= 25:
    Cv = 0.8
elif idade >= 26 and idade <= 40:
    Cv = 0.75
elif idade >= 41 and idade <= 50:
    Cv = 0.65
elif idade >= 51 and idade <= 60:
    Cv = 0.55
elif idade > 60:
    Cv = 0.4

# calcular o novo VPT

VPT_novo = round(Vc * A * Ca * Cl * Cq * Cv, 2)

In [8]:
### taxas de IMI por concelho extraídas do portal das finanças (excepto Gondomar) ###

taxas_imi = {
    '0601' : 0.003,
    '0602' : 0.0038,
    '0603' : 0.003,
    '0604' : 0.003,
    '0605' : 0.004,
    '0606' : 0.0033,
    '0607' : 0.004,
    '0608' : 0.003,
    '0609' : 0.003,
    '0610' : 0.0034,
    '0611' : 0.0033,
    '0612' : 0.003,
    '0613' : 0.003,
    '0614' : 0.00375,
    '0615' : 0.0034,
    '0616' : 0.003,
    '0617' : 0.0045,
    '1901' : 0.003,
    '1902' : 0.003,
    '1903' : 0.003,
    '1904' : 0.003,
    '1905' : 0.003,
    '0101' : 0.003,
    '0102' : 0.003,
    '0103' : 0.003,
    '0104' : 0.003,
    '0105' : 0.004,
    '0106' : 0.003,
    '0107' : 0.0039,
    '0108' : 0.0035,
    '0110' : 0.0033,
    '0111' : 0.003,
    '0112' : 0.0032,
    '0113' : 0.003,
    '0114' : 0.003,
    '0115' : 0.0035,
    '0116' : 0.0035,
    '0109' : 0.00365,
    '0117' : 0.003,
    '0118' : 0.003,
    '0119' : 0.003,
    '0201' : 0.003,
    '0202' : 0.003,
    '0203' : 0.003,
    '0204' : 0.003,
    '0205' : 0.0032,
    '0206' : 0.003,
    '0207' : 0.003,
    '0208' : 0.00375,
    '0209' : 0.00375,
    '0210' : 0.003,
    '0211' : 0.003,
    '0212' : 0.0032,
    '0213' : 0.003,
    '0214' : 0.0032,
    '0301' : 0.003,
    '0302' : 0.0034,
    '0303' : 0.0033,
    '0304' : 0.003,
    '0305' : 0.003,
    '0306' : 0.003,
    '0307' : 0.003,
    '0308' : 0.0033,
    '0309' : 0.003,
    '0310' : 0.003,
    '0311' : 0.003,
    '0312' : 0.0035,
    '0313' : 0.003,
    '0314' : 0.00375,
    '0401' : 0.00395,
    '0402' : 0.003,
    '0403' : 0.003,
    '0404' : 0.003,
    '0405' : 0.003,
    '0406' : 0.003,
    '0407' : 0.003,
    '0408' : 0.003,
    '0409' : 0.003,
    '0410' : 0.003,
    '0411' : 0.003,
    '0412' : 0.003,
    '0501' : 0.004,
    '0502' : 0.003,
    '0503' : 0.0035,
    '0504' : 0.0038,
    '0505' : 0.003,
    '0506' : 0.003,
    '0507' : 0.003,
    '0508' : 0.003,
    '0509' : 0.003,
    '0510' : 0.003,
    '0511' : 0.003,
    '0601' : 0.003,
    '0602' : 0.0038,
    '0603' : 0.003,
    '0604' : 0.003,
    '0605' : 0.004,
    '0606' : 0.0033,
    '0607' : 0.004,
    '0608' : 0.003,
    '0609' : 0.003,
    '0610' : 0.0034,
    '0611' : 0.0033,
    '0612' : 0.003,
    '0613' : 0.003,
    '0614' : 0.00375,
    '0615' : 0.0034,
    '0616' : 0.003,
    '0617' : 0.0045,
    '0701' : 0.0045,
    '0702' : 0.003,
    '0703' : 0.0038,
    '0704' : 0.003,
    '0705' : 0.0042,
    '0706' : 0.003,
    '0707' : 0.003,
    '0708' : 0.0035,
    '0709' : 0.003,
    '0710' : 0.003,
    '0711' : 0.0035,
    '0712' : 0.0034,
    '0713' : 0.003,
    '0714' : 0.0035,
    '0801' : 0.003,
    '0802' : 0.003,
    '0803' : 0.003,
    '0804' : 0.004,
    '0805' : 0.0035,
    '0806' : 0.0036,
    '0807' : 0.0032,
    '0808' : 0.003,
    '0809' : 0.003,
    '0810' : 0.0038,
    '0811' : 0.0042,
    '0812' : 0.0041,
    '0813' : 0.003,
    '0814' : 0.0034,
    '0815' : 0.003,
    '0816' : 0.005,
    '2201' : 0.003,
    '2202' : 0.0032,
    '2203' : 0.003,
    '2204' : 0.003,
    '2205' : 0.003,
    '2206' : 0.003,
    '2207' : 0.003,
    '2208' : 0.003,
    '2211' : 0.003,
    '2209' : 0.003,
    '2210' : 0.003,
    '0901' : 0.003,
    '0902' : 0.003,
    '0903' : 0.004,
    '0904' : 0.003,
    '0905' : 0.0043,
    '0906' : 0.0036,
    '0907' : 0.00375,
    '0908' : 0.003,
    '0909' : 0.003,
    '0910' : 0.003,
    '0911' : 0.003,
    '0912' : 0.0037,
    '0913' : 0.003,
    '0914' : 0.003,
    '2001' : 0.003,
    '2002' : 0.003,
    '2003' : 0.003,
    '2004' : 0.003,
    '2005' : 0.003,
    '2007' : 0.003,
    '2006' : 0.003,
    '1001' : 0.003,
    '1002' : 0.003,
    '1003' : 0.003,
    '1004' : 0.003,
    '1005' : 0.003,
    '1006' : 0.003,
    '1007' : 0.0035,
    '1008' : 0.003,
    '1009' : 0.003,
    '1010' : 0.003,
    '1011' : 0.0045,
    '1012' : 0.0036,
    '1013' : 0.003,
    '1014' : 0.00315,
    '1015' : 0.003,
    '1016' : 0.003,
    '1101' : 0.0038,
    '1115' : 0.003,
    '1102' : 0.00375,
    '1103' : 0.0035,
    '1104' : 0.0036,
    '1105' : 0.0034,
    '1106' : 0.003,
    '1107' : 0.00367,
    '1108' : 0.0035,
    '1109' : 0.0045,
    '1116' : 0.0036,
    '1110' : 0.003,
    '1111' : 0.003,
    '1112' : 0.0039,
    '1113' : 0.0035,
    '1114' : 0.003,
    '2101' : 0.003,
    '2102' : 0.0045,
    '2103' : 0.003,
    '2104' : 0.003,
    '2105' : 0.003,
    '2107' : 0.003,
    '2106' : 0.0045,
    '1201' : 0.003,
    '1202' : 0.003,
    '1203' : 0.003,
    '1204' : 0.003,
    '1205' : 0.003,
    '1206' : 0.003,
    '1207' : 0.004,
    '1208' : 0.003,
    '1209' : 0.003,
    '1210' : 0.003,
    '1211' : 0.0035,
    '1212' : 0.003,
    '1213' : 0.003,
    '1214' : 0.0039,
    '1215' : 0.0038,
    '1301' : 0.003,
    '1302' : 0.003,
    '1303' : 0.003,
    '1305' : 0.003,
    '1306' : 0.00365,
    '1307' : 0.003,
    '1308' : 0.00325,
    '1310' : 0.003,
    '1309' : 0.003,
    '1311' : 0.003,
    '1312' : 0.00324,
    '1313' : 0.003,
    '1314' : 0.00375,
    '1318' : 0.0044,
    '1315' : 0.0038,
    '1316' : 0.003,
    '1317' : 0.0038,
    '1401' : 0.004,
    '1402' : 0.004,
    '1403' : 0.0038,
    '1404' : 0.0039,
    '1405' : 0.0032,
    '1406' : 0.0045,
    '1407' : 0.003,
    '1408' : 0.0035,
    '1409' : 0.0033,
    '1410' : 0.0035,
    '1411' : 0.003,
    '1412' : 0.0035,
    '1413' : 0.003,
    '1421' : 0.0031,
    '1414' : 0.0038,
    '1415' : 0.0035,
    '1416' : 0.00398,
    '1417' : 0.00325,
    '1418' : 0.0035,
    '1419' : 0.0037,
    '1420' : 0.0032,
    '1501' : 0.003,
    '1502' : 0.0037,
    '1503' : 0.0036,
    '1504' : 0.0038,
    '1505' : 0.0034,
    '1506' : 0.0037,
    '1507' : 0.0036,
    '1508' : 0.0034,
    '1509' : 0.0034,
    '1510' : 0.0035,
    '1511' : 0.004,
    '1512' : 0.004,
    '1513' : 0.0034,
    '1601' : 0.0034,
    '1602' : 0.0045,
    '1603' : 0.0032,
    '1604' : 0.003,
    '1605' : 0.003,
    '1606' : 0.0034,
    '1607' : 0.0032,
    '1608' : 0.003,
    '1609' : 0.0036,
    '1610' : 0.003,
    '1701' : 0.0036,
    '1702' : 0.003,
    '1703' : 0.003,
    '1704' : 0.004,
    '1705' : 0.003,
    '1706' : 0.003,
    '1707' : 0.003,
    '1708' : 0.00375,
    '1709' : 0.003,
    '1710' : 0.003,
    '1711' : 0.003,
    '1712' : 0.003,
    '1713' : 0.003,
    '1714' : 0.0039,
    '1801' : 0.003,
    '1802' : 0.003,
    '1803' : 0.003,
    '1804' : 0.003,
    '1805' : 0.00365,
    '1806' : 0.0032,
    '1807' : 0.003,
    '1808' : 0.003,
    '1809' : 0.003,
    '1810' : 0.003,
    '1811' : 0.003,
    '1812' : 0.003,
    '1813' : 0.004,
    '1815' : 0.003,
    '1816' : 0.003,
    '1814' : 0.0039,
    '1817' : 0.003,
    '1818' : 0.003,
    '1819' : 0.004,
    '1820' : 0.003,
    '1821' : 0.003,
    '1822' : 0.003,
    '1823' : 0.003,
    '1824' : 0.003
    }

In [9]:
gondomar = {
    '130401' : 0.00361,
    '130402' : 0.0041,
    '130403' : 0.00361,
    '130404' : 0.0041,
    '130405' : 0.00361,
    '130406' : 0.00361,
    '130407' : 0.00361,
    '130408' : 0.0041,
    '130409' : 0.0041,
    '130410' : 0.0041,
    '130411' : 0.0041,
    '130412' : 0.0041
}

In [10]:
### analisar resultados ###

# ainda não passaram 3 anos após a última avaliação

if data_avaliacao + relativedelta(years=3) > dt.date.today():
    
    print('Ainda não é possível pedir uma reavaliação.')
    print('A última reavaliação deste imóvel foi feita em ' + str(data_avaliacao) + '. A Autoridade Tributária impõe um período mínimo de 3 anos entre reavaliações. Para saber se pode poupar no IMI, volte a esta ferramenta no fim desse prazo, uma vez que vários parâmetros de cálculo podem ser alterados pela Autoridade Tributária até lá.')

# já passaram 3 anos mas o valor patrimonial subiu    

elif VPT_novo > VPT_existente:
    
    print('Não é aconselhável pedir já uma reavaliação.')
    print('Uma reavaliação pedida neste momento irá fazer subir o Valor Patrimonial do imóvel para ' + str(VPT_novo) + ' €. Esta situação poderá dever-se à subida de alguns dos parâmetros de cálculo pela Autoridade Tributária.')

# já passaram 3 anos e pode haver poupança
    
elif VPT_novo <= VPT_existente:
    
    # Gondomar é o único concelho com taxas diferentes em algumas freguesias
    
    if distrito_concelho == '1304':
        taxa_concelho = gondomar[distrito_concelho_freguesia]
    else:
        taxa_concelho = taxas_imi[distrito_concelho]
        
    IMI_existente = round(VPT_existente * taxa_concelho, 2)
    IMI_novo = round(VPT_novo * taxa_concelho, 2)
    
    print('Você pode passar a pagar menos IMI!')
    print('Caso peça uma reavaliação à Autoridade Tributária, o Valor Patrimonial do imóvel passará a ser de ' + str(VPT_novo) + ' € e o valor do IMI anual a pagar de ' + str(IMI_novo) + " €.")
    print('Com a taxa de ' + str(dt.date.today().year) + ', a poupança anual é de ' + str(IMI_existente - IMI_novo) + ' €!')

Ainda não é possível pedir uma reavaliação.
A última reavaliação deste imóvel foi feita em 2020-11-27. A Autoridade Tributária impõe um período mínimo de 3 anos entre reavaliações. Para saber se pode poupar no IMI, volte a esta ferramenta no fim desse prazo, uma vez que vários parâmetros de cálculo podem ser alterados pela Autoridade Tributária até lá.
